# Recommender Systems 
Ramesh Yerraballi for MIS 285N

Content derived from Chapter 22 of Joel Grus book titled, _Data Science from Scratch_(O'Reilly)
In this topic we will look at how website's like Youtube, Netflix, Amazon etc., arrive at customized recommendations that they make to users.

We will look at two Collaborative Filtering alogithms:
1. User-based CF
2. Item-based CF

Setup

In [1]:
from __future__ import division
import math, random
import numpy as np
from collections import defaultdict, Counter

We will look at a simple example where we are given several users and their interests. For example the second user is interested in "NoSQL", "MongoDB", "Cassandra", "HBase", and "Postgres". 

In [2]:
users_interests = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]

## Simplistic Recommendation System: Recommend what is popular
Lets find most commond interests amoung users using a `Counter`.

In [3]:
popular_interests = Counter(interest
                            for user_interests in users_interests
                            for interest in user_interests).most_common()
popular_interests

[('Python', 4),
 ('R', 4),
 ('Big Data', 3),
 ('HBase', 3),
 ('Java', 3),
 ('statistics', 3),
 ('regression', 3),
 ('probability', 3),
 ('Hadoop', 2),
 ('Cassandra', 2),
 ('MongoDB', 2),
 ('Postgres', 2),
 ('scikit-learn', 2),
 ('statsmodels', 2),
 ('pandas', 2),
 ('machine learning', 2),
 ('libsvm', 2),
 ('C++', 2),
 ('neural networks', 2),
 ('deep learning', 2),
 ('artificial intelligence', 2),
 ('Spark', 1),
 ('Storm', 1),
 ('NoSQL', 1),
 ('scipy', 1),
 ('numpy', 1),
 ('decision trees', 1),
 ('Haskell', 1),
 ('programming languages', 1),
 ('mathematics', 1),
 ('theory', 1),
 ('Mahout', 1),
 ('MapReduce', 1),
 ('databases', 1),
 ('MySQL', 1),
 ('support vector machines', 1)]

We can now suggest to a user the most popular interests that are not already in his list

In [4]:
def most_popular_new_interests(user_interests, max_results=5):
    suggestions = [(interest, frequency) 
                   for interest, frequency in popular_interests
                   if interest not in user_interests]
    return suggestions[:max_results]
# Lets get recommendations for second user (index 1)
most_popular_new_interests(users_interests[1],5)

[('Python', 4), ('R', 4), ('Big Data', 3), ('Java', 3), ('statistics', 3)]

In [5]:
print("Most Popular New Interests")
print("already like:", ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"])
print(most_popular_new_interests(["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"]))
print()
print("already like:", ["R", "Python", "statistics", "regression", "probability"])
print(most_popular_new_interests(["R", "Python", "statistics", "regression", "probability"]))
print()

Most Popular New Interests
already like: ['NoSQL', 'MongoDB', 'Cassandra', 'HBase', 'Postgres']
[('Python', 4), ('R', 4), ('Big Data', 3), ('Java', 3), ('statistics', 3)]

already like: ['R', 'Python', 'statistics', 'regression', 'probability']
[('Big Data', 3), ('HBase', 3), ('Java', 3), ('Hadoop', 2), ('Cassandra', 2)]



### What is wrong with this approach?
Recommending something because "lots of people" are interested in it, ignores the user's interests other than to filter out what he is already interested in. 

It would make sense to base the recommendations on what the user actually is interested in!
Enter __Collaborative Filtering (CF) user-based style__ (aka memory-based)

One way to take a user's interests into account is to look for users who are _most similar_ to him and suggest things that those users are interested in.

We need to quantify this term _similarity_, which is done using a couple of different metrics:
1. __Cosine similarity__: If we have two users $i$,$j$ whose interests are the vectors $V_i$ and $V_j$ their cosine similarity is given by the following formula:
$$ Cos(V_i,V_j) = \frac{V_i.V_j}{\sqrt{(V_i.V_i * V_j.V_j)}} $$
where the dot($.$) operator involves performing a dot product of the two vectors. The interest vectors are themselves binary values with $V_i^k$ being $1$ if user $i$ has expressed an interest, in interest $k$ and 0 otherwise.
2. __Pearson-Correlation similarity__: Is a variant of cosine similarity which uses the following formula:
$$ Corr_{(V_i,V_j)} = \frac{(V_i-\bar{V_i}).(V_j -\bar{V_j})}{\sqrt{((V_i-\bar{V_i}).(V_i-\bar{V_i}) * (V_j -\bar{V_j}).(V_j -\bar{V_j}))}}  = Cos((V_i-\bar{V_i}),(V_j -\bar{V_j}))$$

In the following example we will use Cosine similarity

In [15]:
#
# user-based filtering
#
def dot(v,w):
    """ v_1*w_1 + ... v_n*w_n"""
    return sum(v_i *w_i
              for v_i,w_i in zip(v,w))
def cosine_similarity(v, w):
    return dot(v, w) / math.sqrt(dot(v, v) * dot(w, w))

unique_interests = sorted(list({ interest
                                 for user_interests in users_interests
                                 for interest in user_interests }))
unique_interests

['Big Data',
 'C++',
 'Cassandra',
 'HBase',
 'Hadoop',
 'Haskell',
 'Java',
 'Mahout',
 'MapReduce',
 'MongoDB',
 'MySQL',
 'NoSQL',
 'Postgres',
 'Python',
 'R',
 'Spark',
 'Storm',
 'artificial intelligence',
 'databases',
 'decision trees',
 'deep learning',
 'libsvm',
 'machine learning',
 'mathematics',
 'neural networks',
 'numpy',
 'pandas',
 'probability',
 'programming languages',
 'regression',
 'scikit-learn',
 'scipy',
 'statistics',
 'statsmodels',
 'support vector machines',
 'theory']

In [7]:
def make_user_interest_vector(user_interests):
    """given a list of interests, produce a vector whose i-th element is 1
    if unique_interests[i] is in the list, 0 otherwise"""
    return [1 if interest in user_interests else 0
            for interest in unique_interests]
user_interest_matrix = list(map(make_user_interest_vector, users_interests))
print(user_interest_matrix)

[[1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0,

Now we will compute the pair-wise similarities for each user pair.

In [8]:
user_similarities = [[cosine_similarity(interest_vector_i, interest_vector_j)
                      for interest_vector_j in user_interest_matrix]
                     for interest_vector_i in user_interest_matrix]

def most_similar_users_to(user_id):
    pairs = [(other_user_id, similarity)              # find other
             for other_user_id, similarity in         # users with
                enumerate(user_similarities[user_id]) # nonzero
             if user_id != other_user_id and similarity > 0]  # similarity

    return sorted(pairs,                              # sort them
                  key=lambda pair: pair[1],           # most similar
                  reverse=True)                       # first

Lets print users most similar to user 0 and how similar they are

In [9]:
most_similar_users_to(0)

[(9, 0.5669467095138409),
 (1, 0.3380617018914066),
 (8, 0.1889822365046136),
 (13, 0.1690308509457033),
 (5, 0.1543033499620919)]

In [10]:

def user_based_suggestions(user_id, include_current_interests=False):
    # sum up the similarities (of most like users to given user_id) for each interest
    suggestions = defaultdict(float)
    for other_user_id, similarity in most_similar_users_to(user_id):
        for interest in users_interests[other_user_id]:
            suggestions[interest] += similarity

    # convert them to a sorted list
    suggestions = sorted(suggestions.items(),
                         key=lambda pair: pair[1],
                         reverse=True)

    # and (maybe) exclude already-interests
    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                    if suggestion not in users_interests[user_id]]
print("Suggestions for user 0")
user_based_suggestions(0)

Suggestions for user 0


[('MapReduce', 0.5669467095138409),
 ('MongoDB', 0.50709255283711),
 ('Postgres', 0.50709255283711),
 ('NoSQL', 0.3380617018914066),
 ('neural networks', 0.1889822365046136),
 ('deep learning', 0.1889822365046136),
 ('artificial intelligence', 0.1889822365046136),
 ('databases', 0.1690308509457033),
 ('MySQL', 0.1690308509457033),
 ('Python', 0.1543033499620919),
 ('R', 0.1543033499620919),
 ('C++', 0.1543033499620919),
 ('Haskell', 0.1543033499620919),
 ('programming languages', 0.1543033499620919)]

It seems like these are good suggestions for somebody whose interests included "Big-Data". Note that the weights really don't mean anything, they are just used for sorting.

We note that this approach works okay for few items but the notion of similarity between two users looses its meaning when there are a lot of items.


## Item-Based CF
An alternative to user-based CF is to compute the similarities between interests directly. We can then generate suggestions for a user by aggregating interests that are similar to the user's current interests. This is called Item-Based CF

In [11]:
#
# Item-Based Collaborative Filtering
#
# First we will transpose our user_interest_matrix so rows correspond 
# to interests and the columns to users
interest_user_matrix = [[user_interest_vector[j]
                         for user_interest_vector in user_interest_matrix]
                         for j, _ in enumerate(unique_interests)]
interest_user_matrix[0] # tells us that Big_Data is an interest of users 0,8 and 9

[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]

In [12]:
interest_similarities = [[cosine_similarity(user_vector_i, user_vector_j)
                          for user_vector_j in interest_user_matrix]
                         for user_vector_i in interest_user_matrix]

def most_similar_interests_to(interest_id):
    similarities = interest_similarities[interest_id]
    pairs = [(unique_interests[other_interest_id], similarity)
             for other_interest_id, similarity in enumerate(similarities)
             if interest_id != other_interest_id and similarity > 0]
    return sorted(pairs,
                  key=lambda pair: pair[1],
                  reverse=True)

def item_based_suggestions(user_id, include_current_interests=False):
    suggestions = defaultdict(float)
    user_interest_vector = user_interest_matrix[user_id]
    for interest_id, is_interested in enumerate(user_interest_vector):
        if is_interested == 1:
            similar_interests = most_similar_interests_to(interest_id)
            for interest, similarity in similar_interests:
                suggestions[interest] += similarity

    suggestions = sorted(suggestions.items(),
                         key=lambda pair: pair[1],
                         reverse=True)

    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                  if suggestion not in users_interests[user_id]]


In [13]:
print("Suggestions for user 0")
item_based_suggestions(0) # for user 0

Suggestions for user 0


[('MapReduce', 1.861807319565799),
 ('MongoDB', 1.3164965809277263),
 ('Postgres', 1.3164965809277263),
 ('NoSQL', 1.2844570503761732),
 ('MySQL', 0.5773502691896258),
 ('databases', 0.5773502691896258),
 ('Haskell', 0.5773502691896258),
 ('programming languages', 0.5773502691896258),
 ('artificial intelligence', 0.4082482904638631),
 ('deep learning', 0.4082482904638631),
 ('neural networks', 0.4082482904638631),
 ('C++', 0.4082482904638631),
 ('Python', 0.2886751345948129),
 ('R', 0.2886751345948129)]

Again, these are good suggestions for somebody whose interests included "Big-Data". The weights still don't mean anything, they are just used for sorting.

`The End`